<a href="https://colab.research.google.com/github/nosgueira/PLN-2022-1/blob/main/SentimentAnalysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Polaridade Textos

In [ ]:
import pandas as pd
import math
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import numpy as np
from collections import Counter

In [ ]:
df = pd.read_csv('https://raw.githubusercontent.com/viniciusrpb/cic0269_natural_language_processing/main/datasets/corpora/training.csv')
df

,Id,text,Expected
0,0,"Ótima localização, bom atendimento, bom sinal ...",1
1,1,"O hotel é bem localizado e silencioso, o que c...",1
2,2,O hotel é muito bom. O atendimento é excelente...,2
3,3,"Hotel antigo, mal conservado e mal reformado, ...",0
4,4,"Hotel bem localizado, perto de dois shopping, ...",2
...,...,...,...
4995,4995,"Hotel muito bem localizado, com shopping e dem...",2
4996,4996,Além de oferecer uma boa hospedagem com confor...,2
4997,4997,"Hotel charmoso, bom atendimento, quartos confo...",2
4998,4998,"O hotel fica super bem localizado, em frente a...",2


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df.text, df.Expected, test_size=0.10, stratify=df.Expected )

In [ ]:
print(len(X_train))
print(len(X_test))

4500
500


### Representação TFIDF

Gerando representação TFIDF dos dados de treinamento

In [ ]:
vectorizer = TfidfVectorizer()
tfidf_train = vectorizer.fit_transform(X_train).toarray()

In [ ]:
tfidf_test = vectorizer.transform(X_test).toarray()

### K Nearest Neighbours (KNN)

In [ ]:
# Cosine similarity
def cosine_similarity(v1, v2):
    return np.dot(v1, v2) / (np.linalg.norm(v1) * np.linalg.norm(v2))


In [ ]:
# Classify sample based on K neares neighbors
def knn_classify(X_train, y_train, sample_array, K, similarity_function):
  similarities = []
  for vector in X_train:
    similarities.append(similarity_function(sample_array, vector))

  index_similarities = list(zip(similarities, y_train))
  sorted_index_similariteies = sorted(index_similarities, reverse=True)
  nearest_neighbors = [ s for _, s in sorted_index_similariteies[:K]]
  count_labels =Counter(nearest_neighbors)
  return max(count_labels, key=count_labels.get)

### Classificação

In [ ]:
y_pred = [knn_classify(tfidf_train, y_train, sample, 5, cosine_similarity) for sample in tfidf_test]

In [ ]:
target_names = ['negativa', 'neutra', 'positiva']
print(classification_report(y_test, y_pred, target_names=target_names))

              precision    recall  f1-score   support

    negativa       0.48      0.35      0.41        34
      neutra       0.38      0.15      0.21        74
    positiva       0.84      0.95      0.89       392

    accuracy                           0.79       500
   macro avg       0.57      0.48      0.50       500
weighted avg       0.74      0.79      0.76       500

